# Morphisms of A-modules
The ``SteenrodFPModule`` contains functionality to handle modules over the Steenrod algebra, and maps between them.  For a given pair of modules, the set of $A$-linear homomorphisms $\hom(M,N)$ is a graded $\mathbb{F}_p$ vectorspace, and the 

In [2]:
from sage.modules.fp_graded.steenrod.module import SteenrodFPModule as fpmod
A = SteenrodAlgebra(2)
M = fpmod(A, [0], [[Sq(1)], [Sq(2)]])
N = fpmod(A, [0], [[Sq(2)]])

# The set of graded A-linear homomorphisms M -> N
hom = Hom(M,N)
# Get a basis for the degree 18 homomorphisms
hom.basis_elements(n=18)

[Module morphism:
   From: Finitely presented left module on 1 generator and 2 relations over mod 2 Steenrod algebra, milnor basis
   To:   Finitely presented left module on 1 generator and 1 relation over mod 2 Steenrod algebra, milnor basis
   Defn: g[0] |--> Sq(0,1,0,1)*g[0],
 Module morphism:
   From: Finitely presented left module on 1 generator and 2 relations over mod 2 Steenrod algebra, milnor basis
   To:   Finitely presented left module on 1 generator and 1 relation over mod 2 Steenrod algebra, milnor basis
   Defn: g[0] |--> (Sq(5,2,1)+Sq(8,1,1))*g[0]]

## The homspace as a graded vector spce.

The ``extras`` package add methods to the homspace object of morphisms between ``SteenrodFPModules`` so that we can work with this homspace as a vectorspace, one degree at a time.

In [3]:
from fphomspaces import *

# The vectorspace of all degree 18 homomorphisms
hom.vector_presentation(n=18)

Vector space of dimension 2 over Finite Field of size 2

In [4]:
# Given coordinates, we can find the homomorphism it represents
coordinates = (1,0)
f = hom.morphism_from_coordinates(coordinates, n=18)
print(f)

Module morphism:
  From: Finitely presented left module on 1 generator and 2 relations over mod 2 Steenrod algebra, milnor basis
  To:   Finitely presented left module on 1 generator and 1 relation over mod 2 Steenrod algebra, milnor basis
  Defn: g[0] |--> Sq(0,1,0,1)*g[0]


In [5]:
# Given a homomorphism, we can ask for its coordinates (with respect to the basis above)
hom.morphism_to_coordinates(f)

(1, 0)

## Maps of homspaces

In [6]:
M = SteenrodFPModule(A, [0], [[Sq(1)]])
N = SteenrodFPModule(A, [7], [[Sq(1)]])
C = SteenrodFPModule(A, [0], [[Sq(4)+Sq(2)*Sq(2)]])

In [7]:
# Create the homspace of maps between homspaces.

# homhom is a Sage object representing homomorphisms from one
# set of homset of FP-modules to another:

homhom = FPHomspaceMorphismHomspace( Hom(M,C), Hom(N, C) )

# Note: we can't use the usual `Hom` function here, since the 
# `extras` package hasn't been compiled into Sage.

In [8]:
# Choose a map from N to M
f = Hom(N, M).an_element()
f

Module morphism:
  From: Finitely presented left module on 1 generator and 1 relation over mod 2 Steenrod algebra, milnor basis
  To:   Finitely presented left module on 1 generator and 1 relation over mod 2 Steenrod algebra, milnor basis
  Defn: g[7] |--> Sq(0,0,1)*g[0]

In [9]:
# At the moment, the only way to create a map of homspaces is 
# by composition.

# phi is the map Hom(N, C) --> Hom(M, C) given by 
# precomposition by `f`
phi = homhom.precompose(f)

In [10]:
# The map phi of homspaces is a linear transformation of
# vectorspaces
phi.vector_presentation(n=18)

Vector space morphism represented by the matrix:
[1 0 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 0 0]
[1 1 0 0 0]
Domain: Vector space of dimension 5 over Finite Field of size 2
Codomain: Vector space of dimension 5 over Finite Field of size 2

In [11]:
phi.domain() is Hom(M,C)

True

In [12]:
phi.codomain() is Hom(N,C)

True

In [13]:
# There is a utility function that provides a shortcut for the above:
phi_ = f.homspace_map_by_precomposition(C)

## Cohomology
The motivation for treating homspaces as graded vector spaces is to be abe to compute cohomology.

Given a chain complex
$$
    R \xrightarrow{f} S \xrightarrow{g} T\,.
$$
Then for a module $N$, we would like to compute the cohomology of
$$
    \hom(T, N) \xrightarrow{g^*} \hom(S, N) \xrightarrow{f^*} \hom(R, N)\,.
$$

This is accomplished by the new function ``cohomology``.

In [14]:
# Start by computing a resolution of a module.  We do this 
# in this example just to get a pair of composable maps.
A1 = SteenrodAlgebra(2, profile=(2,1))

# Create the coefficients module
F2_A1 = SteenrodFPModule(A1, [0], [[Sq(1)], [Sq(2)]])

res = F2_A1.resolution(4, verbose=False)

In [15]:
# The resolution provides composable maps which we will use to compute cohomology.
f = res[2]
g = res[1]

In [16]:
# The ``cohomology`` function returns the degree ``t`` part of the graded vectorspace
# which is the ker(f^*)/im(g^*):
V, b = cohomology(f, g, F2_A1, t=1)
print('Chomology rank: ', V.dimension())
print('Kernel basis: ', b)

Chomology rank:  1
Kernel basis:  [Module morphism:
  From: Free graded left module on 2 generators over sub-Hopf algebra of mod 2 Steenrod algebra, milnor basis, profile function [2, 1]
  To:   Finitely presented left module on 1 generator and 2 relations over sub-Hopf algebra of mod 2 Steenrod algebra, milnor basis, profile function [2, 1]
  Defn: g[1] |--> g[0]
        g[2] |--> 0]


## Ext-groups

Using the function ``cohomology`` to compute Ext-groups is a little cumbersome.  The new function ``ext`` has a slightly easier interface.

In [17]:
# Computes the F_p-rank of the group Ext^(s,t) (M, N)

# The first parameter is a resolution of M:
ext(res, N=F2_A1, s=1, t=1)

1